In [1]:
"""Задание 1
Необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели
"""

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import r2_score
from sklearn import svm
import matplotlib.pyplot as plt
from matplotlib import colors

file = 'https://raw.githubusercontent.com/Alex-zlat/Kazantcev_DS-25/master/MACHINE_LEARNING/1_Logistic_regression/adult.csv'
data = pd.read_csv(file,parse_dates=True)
data.head()

data.columns = (['age/возраст',                                 # 'Возраст',
                'workclass/рабочий класс',                      # 'Рабочий класс (частный, Self-emp-not-inc, Self-emp-inc, 
                                                                   # федеральное правительство, проч.)', 
                'fnlwgt/кол-во людей',                          # 'final weight) – примерная оценка количества людей, 
                                                                   # которое представляет каждая строка данных',
                'education/образование',                        # 'Образование (бакалавриат, Some-College, 11, HS, Prof-school)',   
                'educational-num/длительность обучения',        # 'Пульс (норм 30-40)',
                'marital-status/семейный статус',               # 'Семейный статус (женат-гражданский-супруг, разведен, 
                                                                   #не был в браке и проч.)', 
                'occupation/род занятий',                       # 'Род занятий (Техподдержка, Ремесло-ремонт, Другое-обслуживание, 
                                                                   # Продажи и проч.)',
                'relationship/родственные связи',               # 'Отношения (Жена, Собственный ребенок, Муж, Не в семье и проч.)'
                'race/Раса',                                    # 'Раса (белая, азиатско-тихоокеанская, американская-индийская-эскимосская, 
                                                                   # другая, черная)',
                'gender/пол',                                   # 'Пол (женский, мужской)'
                'capital-gain/прирост капитала',                # 'Прирост капитала (непрерывная величина)',
                'capital-loss/убыток капитала',                 # 'Убыток капитала (непрерывная величина)'
                'hours-per-week/часов в неделю',                # 'Часов в неделю (непрерывная величина)',
                'native-country/страна происхождения',          # 'Родная страна (США, Камбоджа, Англия и проч.)'
                'income/доход'])                                # 'Доход (> 50K, <= 50K)'
data.tail()

,age/возраст,workclass/рабочий класс,fnlwgt/кол-во людей,education/образование,educational-num/длительность обучения,marital-status/семейный статус,occupation/род занятий,relationship/родственные связи,race/Раса,gender/пол,capital-gain/прирост капитала,capital-loss/убыток капитала,hours-per-week/часов в неделю,native-country/страна происхождения,income/доход
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
48841,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


In [2]:
# исследуем модель зависимости уровня дохода от пола человека, типа занятости и образования
model_1 = LogisticRegression(max_iter=200)  # увеличили количество итераций, чтобы получить сходимость решения 
                                            # (по умолчанию 100 - этого оказывается недостаточно)
le = LabelEncoder() 
select_features = data[['gender/пол', 'workclass/рабочий класс', 'education/образование']] 
X = pd.get_dummies(select_features)   # X = pd.Series(data = le.fit_transform(gender))
income = data['income/доход'].ravel()                                                      

y = pd.Series( data = le.fit_transform( income, ) )                    

# разделим данные на 2 выборки: для обучения (50 процентов) и тестирования обученной модели (50 процентов):
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)



In [3]:
model_1.fit(X_train, y_train)        # обучаем модель model_1.fit(X_train.values.reshape(-1, 1), y_train) 
y_pred = model_1.predict(X_test)     # формируем прогнозные значения  a[:,np.newaxis], y_pred = model_1.predict(X_test.values.reshape(-1, 1))


In [5]:
# смотрим разницу факта и прогноза
result = pd.DataFrame([y_test, y_pred]).T
                      
result.head(8)

,0,1
7762,0.0,0.0
23881,0.0,0.0
30507,1.0,NaN
28911,0.0,NaN
19484,0.0,0.0
43031,0.0,NaN
28188,0.0,NaN
12761,1.0,0.0


In [6]:
# Рассчитаем коэффициент детерминации, демонстрирующий нам уровень ошибки прогноза
# получаем ошибку в половне случаев

model_1.score(X_test, y_test)

0.7865361778796937

In [7]:
# реализуем метод опорных векторов 
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [8]:
y_pred_clf = clf.predict(X_test)         # формируем прогнозные значения в методе опорных векторов 
y_pred_clf

array([0, 0, 0, ..., 0, 0, 0])

In [9]:
# проверим качество прогнозирования 
clf.score(X_test, y_test)

0.7904262724704149

In [10]:
# исследуем модель зависимости уровня дохода от рода занятий и образования
model_2 = LogisticRegression(max_iter=180)
# целевая переменная (столбец 'income/доход') является категориальной
# переведем значения этой колонки в числа, оставив один столбец
le = LabelEncoder() 
select_features = data[['occupation/род занятий', 'education/образование']] 
X = pd.get_dummies(select_features)   
income = data['income/доход'].ravel()                                                     


y = pd.Series( data = le.fit_transform( income, ) )                   

# разделим данные на 2 выборки: для обучения (70 процентов) и тестирования обученной модели (30 процентов):
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
model_2.fit(X_train, y_train)            # обучаем модель 
y_pred = model_2.predict(X_test)         # формируем прогнозные значения

In [12]:
# Рассчитаем коэффициент детерминации, демонстрирующий нам уровень ошибки прогноза
# получаем ошибку в половне случаев

model_2.score(X_test, y_test) 

0.7926704429127142

In [13]:
# реализуем метод опорных векторов для зависимости уровня дохода от заданных признаков 
clf = svm.SVC()
clf.fit(X_train.values, y_train) 

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [14]:
y_pred_clf = clf.predict(X_test)         # формируем прогнозные значения в методе опорных векторов

In [15]:
# проверяем качество прогнозирования 
clf.score(X_test, y_test)  

0.7923292158602334